# 📊 Time Series Analysis - ETS (Error, Trend, Seasonality)

## Overview
This notebook explores ETS (Error-Trend-Seasonality) decomposition for time series analysis.

### What is ETS?
ETS decomposes a time series into three components:
- **Trend (T)**: Long-term progression (increasing/decreasing)
- **Seasonality (S)**: Repeating patterns (daily, monthly, quarterly)
- **Error/Residual (E)**: Random noise after removing trend & seasonality

**Models:**
- **Additive**: Y = T + S + E (when variations are constant)
- **Multiplicative**: Y = T × S × E (when variations grow with level)

### Applications:
- Identify underlying trends
- Detect seasonal patterns (retail sales, tourism)
- Improve forecasting accuracy
- Anomaly detection (residuals analysis)

---

## 1. Setup & Data Loading

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm

%matplotlib inline

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 10)

In [ ]:
# Load airline passengers dataset - classic example with trend & seasonality
# Monthly totals of international airline passengers (1949-1960)
airline = sm.datasets.get_rdataset('AirPassengers', 'datasets').data

# Convert to proper datetime
airline['time'] = pd.to_datetime(airline['time'])
airline = airline.set_index('time')
airline = airline.asfreq('MS')  # Month Start frequency

print("Dataset info:")
print(f"Period: {airline.index.min()} to {airline.index.max()}")
print(f"Frequency: Monthly")
print(f"Length: {len(airline)} observations")
print(f"\nFirst few rows:")
display(airline.head(12))

## 2. Visualize Raw Time Series

In [ ]:
# Plot raw data
fig, ax = plt.subplots(figsize=(14, 6))

airline['value'].plot(ax=ax, linewidth=2, color='steelblue')

ax.set_title('International Airline Passengers (1949-1960)', 
             fontsize=16, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Number of Passengers (thousands)', fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nVisual observations:")
print("✓ Clear upward trend (air travel growth)")
print("✓ Seasonal pattern (peaks in summer months)")
print("✓ Increasing variance over time → Multiplicative model likely better")

## 3. Additive Decomposition

In [ ]:
# Perform additive decomposition
# Y = Trend + Seasonal + Residual
# Use when seasonal variations are roughly constant

decomposition_add = seasonal_decompose(airline['value'], 
                                       model='additive', 
                                       period=12)  # 12 months = 1 year

# Plot decomposition components
fig, axes = plt.subplots(4, 1, figsize=(14, 10), sharex=True)

# Original
decomposition_add.observed.plot(ax=axes[0], color='blue')
axes[0].set_ylabel('Observed', fontsize=11)
axes[0].set_title('Additive Decomposition', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Trend
decomposition_add.trend.plot(ax=axes[1], color='orange')
axes[1].set_ylabel('Trend', fontsize=11)
axes[1].grid(True, alpha=0.3)

# Seasonal
decomposition_add.seasonal.plot(ax=axes[2], color='green')
axes[2].set_ylabel('Seasonal', fontsize=11)
axes[2].grid(True, alpha=0.3)

# Residual
decomposition_add.resid.plot(ax=axes[3], color='red')
axes[3].set_ylabel('Residual', fontsize=11)
axes[3].set_xlabel('Date', fontsize=11)
axes[3].grid(True, alpha=0.3)
axes[3].axhline(y=0, color='black', linestyle='--', linewidth=0.8)

plt.tight_layout()
plt.show()

print("\nAdditive model interpretation:")
print("- Trend: Smooth upward movement")
print("- Seasonal: Fixed amplitude pattern (same peaks/troughs each year)")
print("- Residual: Random noise (should be white noise if model is good)")

## 4. Multiplicative Decomposition

In [ ]:
# Perform multiplicative decomposition
# Y = Trend × Seasonal × Residual
# Use when seasonal variations grow proportionally with trend

decomposition_mult = seasonal_decompose(airline['value'], 
                                        model='multiplicative', 
                                        period=12)

# Plot decomposition components
fig, axes = plt.subplots(4, 1, figsize=(14, 10), sharex=True)

# Original
decomposition_mult.observed.plot(ax=axes[0], color='blue')
axes[0].set_ylabel('Observed', fontsize=11)
axes[0].set_title('Multiplicative Decomposition', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Trend
decomposition_mult.trend.plot(ax=axes[1], color='orange')
axes[1].set_ylabel('Trend', fontsize=11)
axes[1].grid(True, alpha=0.3)

# Seasonal
decomposition_mult.seasonal.plot(ax=axes[2], color='green')
axes[2].set_ylabel('Seasonal', fontsize=11)
axes[2].grid(True, alpha=0.3)

# Residual
decomposition_mult.resid.plot(ax=axes[3], color='red')
axes[3].set_ylabel('Residual', fontsize=11)
axes[3].set_xlabel('Date', fontsize=11)
axes[3].grid(True, alpha=0.3)
axes[3].axhline(y=1, color='black', linestyle='--', linewidth=0.8)

plt.tight_layout()
plt.show()

print("\nMultiplicative model interpretation:")
print("- Trend: Exponential growth pattern")
print("- Seasonal: Proportional variations (grows with trend level)")
print("- Residual: Centered around 1.0 (multiplicative noise)")

## 5. Compare Residuals (Model Quality)

In [ ]:
# Compare residuals to determine which model fits better
# Good residuals should:
# 1. Have mean ≈ 0 (or 1 for multiplicative)
# 2. Have constant variance (homoscedastic)
# 3. Be normally distributed
# 4. Show no autocorrelation (white noise)

fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Additive residuals
axes[0, 0].plot(decomposition_add.resid, color='red', alpha=0.7)
axes[0, 0].set_title('Additive Residuals', fontsize=12, fontweight='bold')
axes[0, 0].axhline(y=0, color='black', linestyle='--')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].hist(decomposition_add.resid.dropna(), bins=30, edgecolor='black', alpha=0.7)
axes[0, 1].set_title('Additive Residuals Distribution', fontsize=12, fontweight='bold')
axes[0, 1].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[0, 1].grid(True, alpha=0.3)

# Multiplicative residuals
axes[1, 0].plot(decomposition_mult.resid, color='blue', alpha=0.7)
axes[1, 0].set_title('Multiplicative Residuals', fontsize=12, fontweight='bold')
axes[1, 0].axhline(y=1, color='black', linestyle='--')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].hist(decomposition_mult.resid.dropna(), bins=30, edgecolor='black', alpha=0.7)
axes[1, 1].set_title('Multiplicative Residuals Distribution', fontsize=12, fontweight='bold')
axes[1, 1].axvline(x=1, color='red', linestyle='--', linewidth=2)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate residual statistics
print("\nResidual Statistics:")
print("="*50)
print(f"\nAdditive Model:")
print(f"  Mean: {decomposition_add.resid.mean():.4f}")
print(f"  Std Dev: {decomposition_add.resid.std():.4f}")

print(f"\nMultiplicative Model:")
print(f"  Mean: {decomposition_mult.resid.mean():.4f}")
print(f"  Std Dev: {decomposition_mult.resid.std():.4f}")

print("\n→ Multiplicative model likely better (smaller, more centered residuals)")

## 6. Extract Seasonal Component for Analysis

In [ ]:
# Extract and visualize seasonal pattern
seasonal_pattern = decomposition_mult.seasonal[:12]  # First 12 months

# Plot seasonal index
fig, ax = plt.subplots(figsize=(12, 6))

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

ax.bar(months, seasonal_pattern.values, color='skyblue', edgecolor='black', alpha=0.7)
ax.axhline(y=1.0, color='red', linestyle='--', linewidth=2, label='Baseline (1.0)')

ax.set_title('Seasonal Pattern - Multiplicative Index', fontsize=14, fontweight='bold')
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Seasonal Factor', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\nSeasonal insights:")
print(f"- Peak travel: {months[seasonal_pattern.values.argmax()]} (summer vacation)")
print(f"- Lowest travel: {months[seasonal_pattern.values.argmin()]} (winter)")
print(f"- Seasonal amplitude: {seasonal_pattern.max() - seasonal_pattern.min():.3f}")

## 7. De-seasonalize & De-trend (Stationarity)

In [ ]:
# Remove seasonality and trend to make series stationary
# Stationary series: constant mean, variance, no trend/seasonality
# Required for ARIMA modeling

# Method 1: Remove trend and seasonal (additive)
detrended_add = airline['value'] - decomposition_add.trend
deseasonalized_add = airline['value'] - decomposition_add.seasonal
stationary_add = decomposition_add.resid

# Plot transformation
fig, axes = plt.subplots(4, 1, figsize=(14, 10), sharex=True)

airline['value'].plot(ax=axes[0], title='Original Series', color='blue')
detrended_add.plot(ax=axes[1], title='Detrended (trend removed)', color='orange')
deseasonalized_add.plot(ax=axes[2], title='Deseasonalized (seasonality removed)', color='green')
stationary_add.plot(ax=axes[3], title='Stationary (trend + seasonality removed)', color='red')

for ax in axes:
    ax.grid(True, alpha=0.3)
    ax.set_ylabel('Value')

axes[3].set_xlabel('Date')
axes[3].axhline(y=0, color='black', linestyle='--', linewidth=0.8)

plt.tight_layout()
plt.show()

print("\nStationarity transformation:")
print("- Original: Non-stationary (trend + seasonality)")
print("- Detrended: Still has seasonality")
print("- Deseasonalized: Still has trend")
print("- Stationary: Ready for ARIMA modeling")

## 8. Key Takeaways

### ETS Decomposition:
1. **Identifies components**: Separates trend, seasonality, noise
2. **Model selection**: Additive vs Multiplicative
3. **Feature engineering**: Extract seasonal patterns
4. **Forecasting**: Predict each component separately

### When to use each model:

**Additive (Y = T + S + E)**:
- ✅ Constant seasonal variations
- ✅ Stable variance over time
- Example: Temperature data

**Multiplicative (Y = T × S × E)**:
- ✅ Seasonal variations grow with level
- ✅ Increasing variance (heteroscedastic)
- Example: Sales, airline passengers, stock prices

### Applications in Finance:
- Retail sales forecasting
- Revenue prediction (quarterly seasonality)
- Tourism demand modeling
- Commodity prices (harvest cycles)

### Next Steps:
- Use ARIMA for forecasting the stationary residuals
- Combine with external regressors (ARIMAX)
- Test for structural breaks

---
*Next: ARIMA modeling for forecasting*